In [ ]:
# Outiers, z-score future desription demo idea

In [2]:
import pandas as pd
import numpy as np

In [9]:
item = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')
sales_train = pd.read_csv('sales_train.csv')
categories = pd.read_csv('item_categories.csv')

In [12]:
sales_train["date"] = pd.to_datetime(sales_train["date"], errors="coerce")

# Predecting future sales

In [20]:
sales_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,1.043246e+06,1.043246e+06,1.043246e+06,1.043246e+06,1.043246e+06
mean,4.472810e+00,3.245368e+01,1.024260e+04,6.823654e+02,1.220174e+00
std,2.967065e+00,1.625856e+01,6.305360e+03,1.124829e+03,1.600203e+00
min,0.000000e+00,0.000000e+00,1.200000e+01,-1.000000e+00,-6.000000e+00
25%,2.000000e+00,2.200000e+01,4.548000e+03,1.990000e+02,1.000000e+00
50%,4.000000e+00,3.000000e+01,9.691000e+03,3.990000e+02,1.000000e+00
75%,7.000000e+00,4.700000e+01,1.561875e+04,6.993000e+02,1.000000e+00
max,1.000000e+01,5.900000e+01,2.216800e+04,5.920000e+04,2.880000e+02


In [13]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043246 entries, 0 to 1043245
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   date            1043246 non-null  datetime64[ns]
 1   date_block_num  1043246 non-null  int64         
 2   shop_id         1043246 non-null  int64         
 3   item_id         1043246 non-null  int64         
 4   item_price      1043246 non-null  float64       
 5   item_cnt_day    1043246 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 47.8 MB


In [16]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1
1,2013-03-01,0,25,2552,899.00,1
2,2013-05-01,0,25,2552,899.00,-1
3,2013-06-01,0,25,2554,1709.05,1
4,2013-01-15,0,25,2555,1099.00,1


In [15]:
sales_train["item_id"].value_counts()

20949    6702
1830     3475
2808     3345
6740     2782
2445     2755
         ... 
3681        1
3999        1
18838       1
5218        1
2305        1
Name: item_id, Length: 14168, dtype: int64

In [17]:
sales_train.isnull()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
1043241,False,False,False,False,False,False
1043242,False,False,False,False,False,False
1043243,False,False,False,False,False,False
1043244,False,False,False,False,False,False


In [19]:
sales_train.index

RangeIndex(start=0, stop=1043246, step=1)

In [6]:
item.describe()  # no purpose of doing this as this data doesn't explains anything

,item_id,item_category_id
count,22170.00000,22170.000000
mean,11084.50000,46.290753
std,6400.07207,15.941486
min,0.00000,0.000000
25%,5542.25000,37.000000
50%,11084.50000,40.000000
75%,16626.75000,58.000000
max,22169.00000,83.000000


In [ ]:
# Outlier: An outlier is an observation that lies an abnormal distance from other values in a random sample from
# a population

# Two graphical techniques for identifying outliers, scatter plots and box plots, along with an analytic procedure
# for detecting outliers when the distribution is normal (Grubbs' Test)
# 

In [29]:
item_id_price = sales_train.loc[:, ["item_price", "item_cnt_day"]]

In [31]:
(sales_train.loc[:, ["item_price", "item_cnt_day"]]).describe()

,item_price,item_cnt_day
count,1.043246e+06,1.043246e+06
mean,6.823654e+02,1.220174e+00
std,1.124829e+03,1.600203e+00
min,-1.000000e+00,-6.000000e+00
25%,1.990000e+02,1.000000e+00
50%,3.990000e+02,1.000000e+00
75%,6.993000e+02,1.000000e+00
max,5.920000e+04,2.880000e+02


In [30]:
item_id_price.describe()

,item_price,item_cnt_day
count,1.043246e+06,1.043246e+06
mean,6.823654e+02,1.220174e+00
std,1.124829e+03,1.600203e+00
min,-1.000000e+00,-6.000000e+00
25%,1.990000e+02,1.000000e+00
50%,3.990000e+02,1.000000e+00
75%,6.993000e+02,1.000000e+00
max,5.920000e+04,2.880000e+02


In [33]:
sales_train[["item_price", "item_cnt_day", "item_id"]]

,item_price,item_cnt_day,item_id
0,999.00,1,22154
1,899.00,1,2552
2,899.00,-1,2552
3,1709.05,1,2554
4,1099.00,1,2555
...,...,...,...
1043241,199.00,1,18474
1043242,199.00,1,18474
1043243,199.00,1,18484
1043244,99.00,1,19751


In [34]:
(sales_train[["item_price", "item_cnt_day", "item_id"]]).describe()

,item_price,item_cnt_day,item_id
count,1.043246e+06,1.043246e+06,1.043246e+06
mean,6.823654e+02,1.220174e+00,1.024260e+04
std,1.124829e+03,1.600203e+00,6.305360e+03
min,-1.000000e+00,-6.000000e+00,1.200000e+01
25%,1.990000e+02,1.000000e+00,4.548000e+03
50%,3.990000e+02,1.000000e+00,9.691000e+03
75%,6.993000e+02,1.000000e+00,1.561875e+04
max,5.920000e+04,2.880000e+02,2.216800e+04


In [40]:
sales_train[["item_price", "item_cnt_day", "item_id"]].dtypes

item_price      float64
item_cnt_day      int64
item_id           int64
dtype: object